<h1 style="color:black" align="center">Предобрадотка данных</h1>

<h1 style="color:#008B8B">3. Обработка категориальных признаков</h1>

<h2 style="color:#008B8B">3.2 One-hot encoding</h2>

Если у признака имеется много категорий и среди них имеются редкие, тогда возникает множество проблем.

Решается данная проблема следующим образом:

* **Trash bucket** - соеденяем редкие категории в одну.

* **Хеширование** - делаем хеш функцию h, которая множество категорий признака переводит в другое множество, которое меньше по своей мощности.

$\large C = \{c_1, \ldots, c_m\}$ - множество значений признака

$\large h: C \to \{1, \ldots, K\}$ где $K < |C|$

Если правильно подобрать $K$, тогда соеденение признаком посредством хеширования не испортит модель, новый признак будет информативен так же, как они в раздельности. После хеширования могут возникнуть коллизии, но они не будут оказывать негативное воздействие.

**Random projections** 

Имеется набор признаков $x$. Гененрируем набор весов из нормлаьного распределения $w \sim \mathcal{N}(0, 1)$, после добавляем новый признак $\langle w, x \rangle$. И так можно повторить много раз, если у нас есть миллион признаков, мы генерируем много векторов весов такой же размерности и заменяем старые признаки на скалярное произведение вектора весов на старые признаки.

Данный метод позвоялет понижаеть размерность без большого влияния на качество модели.

<h2 style="color:#008B8B">3.3 Кодирование с учётом целевой переменной</h2>

### Для бинарной классификации

$\large \mathbb{Y} = \{1, 0\}$

$\large g_j(x, X) = \frac{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)]}$

где $f_j(x_i)$ - $j$-й признак $i$-го объекта, $y_i$ - класс $i$-го объекта.

Здесь мы заменяем $j$-й категориальный признак на числовой. В числителе мы считаем количество объектов положительного класса, в знаменателе общее количество объектов признака, значит, мы получим долю объектов положитлеьного класса. Для отрицательного класса, мы можем найти долю как $1 - g_j$. 

Мы получаем для каждой категории вероятность, того что объект с этой категорией относится к положительному классу.

Если у некоторого признака все объекты категории положитлеьные, это указывает на то, что скорее всего объект данной категории будет положительным.

### Для многоклассовой классификации

$\large \mathbb{Y} = \{1, \ldots, K\}$

Отметим, что эту формулу легко перенести как на случай многоклассовой классификации (в этом случае будем считать K признаков, по одному для каждого класса, и в числителе будет подсчитывать долю объектов с заданной категорией и с заданным классом), так и на случай регрессии (будем вычислять среднее значение целевой переменной среди объектов данной категории).

$\large g_{jk}(x, X) = \frac{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)][y_i = k]}{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)]}$

В числителе суммируем по всем объектам, ищем объекты с такой же категорией как у $\ell$ объекта и средни них считаем объекты только $k$-го класса. И делим на количество объектов данной категории. Тем самым, мы каждую категорию кодируем распределением на $K$ исходов, которое говорит о том, какая вероятность при некой категории получить свой класс.

### Для регрессии

$\large \mathbb{Y} = \mathbb{R}$

Мы считаем среднее значение целевой переменной внутри данной категории.

$\large g_{j}(x, X) = \frac{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)]y_i}{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)]}$


### Переобучение

Напряму считать новый признак под данным формулам будет плохой идеей, так как модель может переообучиться. Если у некоторой категории признака целевая переменная будет все время принимать один класс (у данной категории оказалось, например, два объекта для которых целева переменная совпала), тогда значение $g_j$ будет равно значению целевой переменной. Тем самым, модель может подумать что в столбец стоит ответ. Так как значение целевой переменной было оценено малым количеством объектов, тогда данная оценка будет ненадежной, смешненной и так как модель доверяет данной оценке, она может переобучиться. 

### Борьба с переобучением

И нам необходимо, чтобы модель могла опираться на значение счетчика только в случае высокой уверенности того, что оценка внутри данной категории скорее всего соответствует классу целевой переменной.

$\large g_j(x, X)
=
\frac{
    \sum_{i=1}^{\ell}
        [f_j(x) = f_j(x_i)][y_i = +1]
        +
        \frac{C}{\ell}
        \sum_{i=1}^{\ell}
        [y_i = +1]
}{
    \sum_{i=1}^{\ell}
    [f_j(x) = f_j(x_i)] + C
}$

где $C$ – коэффициент, отвечающий за баланс между средним значением по категории и глобальным средним значением (гиперпараметр).

Предаоложим, что C = 100 и всего один объект в некоторой категории. Если значение целевой переменной равно 1, тогда классическая формула $g_j$ (без правой) вернет 1, так как в числителе и знаменателе будут 1. Получается что правая часть будет оказывать воздействи на результат:

$\large \frac{C}{\ell} \sum_{i=1}^{\ell} [y_i = +1]$ - доля положительный объектов в выборке умноженное на $C$. 

Так как в знаменателе мы делим на $C$, тогда $C$ просто сокращаются и данная дробь будет близка к доле положительных объектов в выборке. Чем больше в категории будет объектов с положительной целевой переменной (две суммы слева), тем больше влияение на результут дроби они будут оказывать, так как поправка $C$ будет вносить меньший вклад.

Если мало объектов в данной категории, тогда $g_j(x, X) \approx \frac{1}{\ell} \sum_{i=1}^{\ell} [y_i = +1]$ будет примерно равно доли положительных объектов в выборке. Тогда модель не сможет переобучиться под такое значение, так как доля положительных объектов в выборке не содержит информации о объекте и модель не сможет узнать ответ на конкретном объекте.

А если объекту много, тогда $g_j(x, X) \approx \text{классической формуле}$.

### Почему выходы других моделей приводят к переобучению?

$g_j(x, X)$ является простой моделью, так как это число посчитанное с учетом целевой переменной. И тогда $a(x)$, которая обучается на новых признаках будет иметь выходы другой простой модели. 

Почему если среди признаков есть выходы другой модели? Если мы уверены что признаки имееют одинаковый смысл как обучающей так и на тестовой выборке, например, цвет автомобиля там и там будет иметь один смысл. Но этот признак на основе выхода модели может изменить свой смысл и если модель $g_j(x, X)$ будет переобучена, тогда качество на обучающей выборке будет выше чем на тестовой. 

Когда модель будет обучаться на обучающей выборке, модель заметит что признак оказывает большое влияние на предсказание и модель будет на него пологаться. Но когда модель будет применена на тестовой выборке, модель прежде будет уверена что это признак является хорошим, но так как выход простой модели изменил смысл признака. Почему смысл признака был изменен? Так как мы обучали его на обучающей выборке с известными целевыми переменными, а на тренировочной выборке нам целевые переменные неизвестны, тогда мы заменяем категории признаков используя уже обученную модель, котора могла подогнаться под обучающую выборку. Именно поэтому мы подбираем $C$ по тренировочной выборке.

### Сглаживание

Можно немного модифицировать формулу,
чтобы сила регуляризации зависела от объёма данных по конкретной категории:


$ g_j(x, X)
=
\lambda \left( n(f_j(x)) \right)
\frac{
    \sum_{i=1}^{\ell}
        [f_j(x) = f_j(x_i)][y_i = +1]
}{
    \sum_{i=1}^{\ell}
    [f_j(x) = f_j(x_i)]
}
+
\left( 1 - \lambda \left( n(f_j(x)) \right) \right)
\frac{1}{\ell}
\sum_{i = 1}^{\ell}
    [y_i = +1]$

где $n(z) = \sum_{i = 1}^{\ell} [f_j(x_i) = z]$ - число объектов категории $z$, 

$n(f_j(x))$ - число объектов данной категорией в выборке.

$\lambda(n)$ - некоторая монотонно возрастающая функция, дающая значения из отрезка $[0, 1]$.

Примером может служить $\lambda(n) = \frac{1}{1 + \exp(-n)}$.
Если грамотно подобрать эту функцию, то она будет вычищать значение целевой переменной из редких категорий и мешать переобучению. 

Если категория будет большой по размеру функция будет отдавать предпочтени первому слогаемому, если маленькая тогда второму. Идея заключается в том, что чем меньше значений в категории, тем значение счетчика будет больше сдвигать к среднему значению класса по всей выборке.

### Зашумление

Можно посчитать новые признаки по базовой формуле (3.1), а затем просто добавить к каждому значению случайный шум (например, нормальный). Это
действительно снизит уровень корреляции счётчиков с целевой переменной. Проблема в том, что это делается за счёт снижения силы такого признака, а значит, мы ухудшаем итоговое качество модели. Поэтому важно подбирать дисперсию шума, чтобы соблюсти баланс между борьбой с переобучением и предсказательной силой счётчиков.

$\large \tilde{g}_{j}(x, X) = g_{j}(x, X) +  \mathcal{N}(\mu, \sigma^2)$,

где  $\mathcal{N}(\mu, \sigma^2)$ — нормальное распределение
со средним $\mu = 0$ и дисперсией $\sigma^2$. 


Здесь необходимо правильно подобрать $\sigma^2$, так чтобы на маленьких категориях сильно зашумлял признак, а на больших не очень, а это возможно если $\sigma^2 (f_j(x))$ зависит от категорий на данном объекте, что весьма сложная задача.
